# Trabalho Final de Estágio Docente
## Estagiário: Douglas Macedo Sgrott
## Aluno: Vinicius Gasparini
## Data de entrega: 21/06/2021 (segunda-feira)
## O trabalho está organizado em partes:
 - ### **Dataset: Onde você irá limpar e pre processar o dataset. Atribua a versão final do dataset em um dataframe chamado df.**
 - Separação dos dados: Aqui os dados são normalizados e divididos em Treino/Validação. Não precisa modificar o código.
 - ### **Arquitetura da Rede Neural: Onde você vai definir a arquitetura da rede neural.**
 - ### **Parâmetros de otimização da Rede Neural: Onde você vai definir outros parâmetros da rede neural.**
 - Visualização dos resultados: Onde os resultados são obtidos
 - Exemplos: Servir como exemplo de análise, data cleaning e pré-processamento.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# importing stuff
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# a lot of stuff
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

# Z-score / outliers stuff
from scipy import stats

# Rede Neural stuff
from tensorflow.keras import regularizers, callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import plot_model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset
#### Coloque aqui seu data cleaning e seu pre-processamento e atribua o dataset para um dataframe chamado **df**

In [ ]:
df = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col=0)

# Analizando as features para auxiliar na caracterização
features = list(df.head(0))
sorted({feat:df[feat].nunique() for feat in features}.items(),key=lambda item: item[1])

Iremos definir como variáveis categóricas `animal, furniture` e `city`

Vamos dar uma olhada nos tipos de dado padrão do nosso dataframe

In [ ]:
categorical_features = ['animal', 'furniture', 'city']
df.info()

Olhando para o tipo das variáveis vemos que `floor` esta como `object`, isso pode indicar alguma inconsistência nos dados

In [ ]:
# Listando os valores únicos
floor_unique = df.floor.unique()
print(f"{len(floor_unique)} valores unicos.\nSendo eles {list(floor_unique)}")

Vemos que existe o valor `'-'`, vamos alterar estes e os não informados para `0` e assim manter consistência nos dados

In [ ]:
df.floor.replace(to_replace='-',value='0',inplace=True)
df.floor.fillna(value='0',inplace=True)
df.floor = pd.to_numeric(df['floor'], downcast='float')

# Listando os valores únicos
floor_unique = df.floor.unique()
print(f"{len(floor_unique)} valores unicos.\nSendo eles {list(floor_unique)}")

### Analizando dados inválidos ou ausentes

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
pd.DataFrame({'column_name': df.columns, 'percent_missing': percent_missing})

Para que nossa análise seja mais precisa, vamos imputar alguns dados nas colunas `furniture`e `animal`

Em `furniture` iremos considerar que se não foi informado, isso corresponde ao valor `not furnished`. Quanto a `animal`, será criado um terceiro valor, `unknown` para quando não houve especificação.

In [ ]:
df.furniture.fillna('not furnished',inplace=True)
df.animal.fillna('unknown',inplace=True)

# Checando as alterações
percent_missing = df.isnull().sum() * 100 / len(df)
pd.DataFrame({'column_name': df.columns, 'percent_missing': percent_missing})

Uma vez com esses valores preenchidos, conseguimos prosseguir com a análise dos dados faltantes por linha

In [ ]:
# Gerando histograma para analisar a quantidade de ocorrências de dados ausentes por linha
df_hist_missing = pd.DataFrame()
for col in df.columns:
    missing = df[col].isnull()
    miss_count = np.sum(missing)
    if miss_count > 0:  
        df_hist_missing['{}_miss_count'.format(col)] = missing


miss_count_cols = df_hist_missing.columns
df_hist_missing['miss_count'] = df_hist_missing.sum(axis=1)
df_hist_missing['miss_count'].value_counts().reset_index().sort_values(by='index').plot.bar(x='index', y='miss_count')

In [ ]:
# Sob posse dessas informações, vamos então remover as linhas cujo falta de dados ultrapasse 2 colunas

ind_missing = df[df_hist_missing['miss_count'] > 2].index
df.drop(ind_missing, axis=0, inplace=True)

Iremos utilizar uma modelagem de regressão para predizer alguns dados de `rent amount, area, property tax e total`

In [ ]:
# Criamos um dataframe com os dados de rent amount (R$), area, property tax (R$) e total (R$)
df_regress = pd.concat([df['rent amount (R$)'], df['area'], df['property tax (R$)'], df['total (R$)']], axis=1)
df_regress.head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_regress.values)

# Agora, faremos uma regressão nos mesmos dados usados no treinamento, para
# gerar valores numéricos para substituir os valores ausentes de LotFrontage
X = df_regress.values
regr_output = imp_mean.transform(X)
pd.DataFrame(regr_output).describe()

Aparentemente a regressão foi bem sucedida, vamos incorporar esses dados ao nosso dataframe com excessão do `total (R$)`

In [ ]:
df['rent amount (R$)'] = regr_output[:, 0]
df['area'] = regr_output[:, 1]
df['property tax (R$)'] = regr_output[:, 2]

Vamos agora preencher os dados de `city` utilizando a estratégia de copiar o valor do registro anterior

In [ ]:
df['city'].fillna(method='ffill',inplace=True)

Irei retirar a coluna `hoa (R$)` e preencher os dados de `fire insurance (R$)` com valores da média

Dados como `rooms, bathroom, parking spaces` serão preenchidos pela mediana.

In [ ]:
features.remove('hoa (R$)')
df = df.loc[:, features]

df['fire insurance (R$)'] = df['fire insurance (R$)'].fillna(df['fire insurance (R$)'].mean())

df['rooms'] = df['rooms'].fillna(df['rooms'].median())
df['bathroom'] = df['bathroom'].fillna(df['bathroom'].median())
df['parking spaces'] = df['parking spaces'].fillna(df['parking spaces'].median())

### Detecção de outliers

In [ ]:
def features_boxplot():
    selected_features = [feature for feature in df.columns if feature not in categorical_features]
    df_without_missing = df.dropna()
    fig, axes = plt.subplots(ncols=len(selected_features), figsize=(20, 5))

    for i,col in enumerate(selected_features):
        axes[i].boxplot(df_without_missing[col])
        axes[i].set_title(col)

    plt.tight_layout()
features_boxplot()

Vemos claramente que há um `outlier` em `floor`

In [ ]:
print("Outlier:", df.floor.max())
df.drop(df.loc[df['floor'] == df.floor.max()].index, inplace=True)

print("Novo boxplot")
df_without_missing = df.dropna()
fig, axes = plt.subplots(ncols=1, figsize=(3, 5))

axes.boxplot(df_without_missing['floor'])
axes.set_title('floor')

plt.tight_layout()

Vemos que possuimos dois boxplot muito semelhantes, o de `total (R$)` e o de `area`

Vamos então plotar um gráfico de dispersão para tentar analisar a relação entre essas duas features

In [ ]:
def graph_scatter(x,y):
    fig, ax = plt.subplots()

    ax.scatter(x=df[x], y=df[y])
    ax.set_ylabel(x)
    ax.set_xlabel(y)
    plt.show()
graph_scatter("area","total (R$)")

Analisando este gráfico, iremos filtrar:
* area >= 3000
* total (R$) >= 0.4e6

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['area'] < 3000
df = df[mask]
mask = df['total (R$)'] < 0.4e6
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter("area","total (R$)")

Podemos refinar ainda mais diminuindo nosso filtro de area para 1250 e total (R$) para 30000

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['area'] < 1250
df = df[mask]
mask = df['total (R$)'] < 3e4
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter("area","total (R$)")

Após essa correções, vamos olhar novamente para os boxplot

In [ ]:
features_boxplot()

Como último passo, iremos normalizar os dados categóricos para valores numéricos

In [ ]:
df['city'] = pd.Categorical(df.city)
df['city'] = df.city.cat.codes
df['city'] = pd.to_numeric(df['city'], downcast='float')

df['animal'] = pd.Categorical(df.animal)
df['animal'] = df.animal.cat.codes
df['animal'] = pd.to_numeric(df['animal'], downcast='float')

df['furniture'] = pd.Categorical(df.furniture)
df['furniture'] = df.furniture.cat.codes
df['furniture'] = pd.to_numeric(df['furniture'], downcast='float')

df.head()

## Novas features

Serão criadas 4 novas features

### Razão entre $\frac{area}{rooms}$
### Valor $\frac{rent\ amount\ (R\$)}{rooms + bathroom}$
### $area^2$
### $rent\ amount\ (R\$) + property\ tax\ (R\$) + fire\ insurance\ (R\$)$


In [ ]:
df['area_per_room'] = df.area / df.rooms 
df['rent_per_ambient'] = df['rent amount (R$)'] / (df.rooms + df.bathroom)
df['area_squared'] = df.area ** 2
df['sum_taxes'] = df['rent amount (R$)'] + df['property tax (R$)'] + df['fire insurance (R$)']


# Separação dos dados

In [ ]:
# Normalizamos os dados de df em uma escala de [0, 1]
# Estou fazendo isto aqui pois temos que "desnormalizar" na hora de gerar os gráficos de R²
column_names = df.columns
scaler = MinMaxScaler()
scaler.fit(df)
df = scaler.transform(df)
df = pd.DataFrame(df)
df.columns = column_names

# Pegamos o dataset df e separamos em x (entrada) e y (saida), numa separação 70% treino e 30% validação
input_dim = df.shape[1] - 1
x = df.drop(columns='total (R$)')
y = df['total (R$)']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.30, random_state=42)

# Arquitetura da Rede Neural
#### Criei um código bem simples pra permitir criar diferentes redes neurais modificando apenas algumas variáveis (EM CAPSLOCK),
#### Mas se quiser criar sua própria arquitetura mais customizada, fique a vontade

In [ ]:
NEURONIOS_CAMADA_INICIAL = 20

# Número de camadas intermediárias e neurônios. Tamanho do array são os números de camadas, elementos do array são números de neurônios.
# Ex: [30, 15] = 2 camadas intermediárias com 30 neurônios na primeira e 15 neurônios na segunda
# Ex: [] = Nenhuma camada intermediária
# Ex: [10, 10, 10, 10, 50] = 5 camadas intermediárias, com 10 neurônios nas 4 primeiras e 50 neurônios na última
NEURONIOS_CAMADAS_INTERMEDIARIAS = [10, 5]

# Usar dropout: True para usar, False para não usar
USAR_DROPOUT = False

# Porcentagem de Dropout: valor entre 0 e 1
DROPOUT_VALUE = 0.2

# Regularizador: None = Não usar regularizador, 'l1' = Reg L1, 'l2' = Reg L2
TIPO_REGULARIZADOR = None

# Função de ativação: 'relu', 'tanh', 'sigmoid', 'softmax', 'softplus', 'elu'
FN_ATIVACAO = 'elu'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


# "Doug, mas que código tosco!" Também acho... Caso você queira criar sua própria arquitetura
# sem usar os parâmetros acima, é bem simples. Segue abaixo um exemplo meio doideira:
# model = Sequential()
# model.add(Dense(units=30, input_dim=input_dim, activation='relu', kernel_regularizer='l1'))
# model.add(Dropout(0.4, input_shape=(30,)))
# model.add(Dense(units=20, activation='tanh', kernel_regularizer='l2'))
# model.add(Dense(units=20, activation='relu', kernel_regularizer=None))
# model.add(Dense(units=1, activation='relu'))

plot_model(model, show_shapes=True)

# Parâmetros de otimização da Rede Neural
#### Pode alterar os valores das variáveis que estão EM CAPSLOCK

In [ ]:
CALLBACKS = [callbacks.TerminateOnNaN()] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_squared_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 50
OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!

# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)


# Visualização dos resultados

In [ ]:

df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = plt.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

plt.show()